In [128]:
import os

USER_FILE = "users.txt"
ACCOUNTS_FILE = "accounts.txt"
TRANSACTION_FILE = "transactions.txt"

In [129]:
def main():
    while True:
        print('''
1 > Continue to Login
2 > Exit
''')
        option = input('Welcome to Banking System. Do you want to Continue? (Y to Continue) (N to Exit)').lower()

        try:
            if option == '1' or option == 'y' or option == 'login':
                login()
            elif option == '2' or option == 'n' or option == 'exit' or option == 'e':
                print('Quiting System.')
                break
            else:
                print('Invalid option. Please select a valid operation.')
            
        except ValueError:
            print('Invalid input. Please enter a valid input.')

In [130]:
class User:
    def __init__(self, user_name, user_type):
        self.__user_name = user_name
        self.__user_type = user_type

    def get_user_name(self):
        return self.__user_name
    
    def get_user_type(self):
        return self.__user_type

In [131]:
def authenticateUser(user_name_arg, password_arg):
    if not os.path.exists(USER_FILE):
        print("User list file not found.")
        return
    
    with open(USER_FILE, "r") as file:
        users = file.readlines()
        
        for user in users:
            user_name, password, user_type = user.strip().split(",")
            if user_name == user_name_arg and password == password_arg:
                return User(user_name, user_type)
        print('Invalid credentials. No user found!')
        return None

In [132]:
def login(): 
    user_name = input('Enter UserName: ')
    password = input('Enter Password: ')
    user = authenticateUser(user_name, password)
    if not user == None:
        if user.get_user_type().lower() == 'admin':
            adminRoles(user)
        elif user.get_user_type().lower() == 'user':
            userRoles(user)
        else:
            print(f'Invalid user type. No supported operation for user type {user.get_user_type()}.')
    else:
        return

In [147]:
def userRoles(user):
    while True:
        print(f'''
/Dashboard
***** Hello {user.get_user_name()} - {user.get_user_type().upper()} *****
1 > Check Balance
2 > Deposit Amount
3 > Withdraw Amount
4 > Statement
5 > Exit
''')
        try:
            operation = input('Select what operation to perform: ')
            
            if operation == '1':
                print('**** /Dashboard/Check Balance ****')
                account_no_filter = input('Enter your account number: ')
                account_data = load_user(account_no_filter)
                if account_data is not None:
                    print(f'Account Title: {account_data["account_title"]} has ${account_data["balance"]} balance in account. ')
            
            elif operation == '2':
                print('**** /Dashboard/Deposit Amount ****')
                account_no_filter = input('Enter your account number: ')
                account_data = load_user(account_no_filter)
                if account_data is not None:
                    amount = float(input('Enter amount to deposit: '))
                    deposit(account_data, amount)
            
            elif operation == '3':
                print('**** /Dashboard/Withdraw Amount ****')
                account_no_filter = input('Enter your account number: ')
                account_data = load_user(account_no_filter)
                if account_data is not None:
                    amount = float(input('Enter amount to deposit: '))
                    withdraw(account_data, amount)
            
            elif operation == '4':
                print('**** /Dashboard/Statement ****')
                account_no_filter = input('Enter your account number: ')
                account_data = load_user(account_no_filter)
                if account_data is not None:
                    load_statement(account_data)
                    
            elif operation == '5':
                print('Exit')
                break
                
            else:
                print('Invalid option. Please select a valid operation.')
        
        except ValueError:
            print('Invalid input. Please enter a valid input.')

In [134]:
def adminRoles(user):
    while True:
        print(f'''
/Dashboard
***** Hello {user.get_user_name()} - {user.get_user_type().upper()} *****
1 > Create New User Account
2 > View total deposits in the bank. 
3 > View total withdrawal in the bank. 
4 > Check the total number of accounts.
5 > Exit
''')
        try:
            operation = input('Select what operation to perform: ')
            
            if operation == '1':
                print('**** /Dashboard/Create New User Account ****')
                account_user_name = input('Enter the User Name: ')
                account_title = input('Enter the Account Title: ')
                account_number = input('Enter the Account Number: ')
                create_account(account_title, account_number, account_user_name)
            
            elif operation == '2':
                print('**** /Dashboard/View total deposits in the bank. ****')
                load_deposit_transactions()

            elif operation == '3':
                print('**** /Dashboard/View total withdrawal in the bank. ****')
                load_withdrawal_transactions()
            
            elif operation == '4':
                print('**** /Dashboard/Check the total number of accounts. ****')
                load_accounts()
                        
            elif operation == '5':
                print('Exit')
                break
                
            else:
                print('Invalid option. Please select a valid operation.')
        
        except ValueError:
            print('Invalid input. Please enter a valid input.')

In [135]:
# Create an account
def create_account(account_title, account_number, user_name ,initial_balance=0.0):
    if (account_title == '' or account_number == ''):
        print('ERROR: Account title and Account number is required field to create account')
        return
    account = {
        "user_name": user_name,
        "account_title": account_title,
        "account_number": account_number,
        "balance": initial_balance,
        "transactions": []
    }
    record_user(account)
    print(f"====== Congratulations! Account for {account_title} created with balance ${initial_balance}. ======")
    return account

In [136]:
# Record User to file.
def record_user(account):
    with open(USER_FILE, "a") as file:
        file.write(f"{account['user_name']},{account['user_name']},user\n")
        
    with open(ACCOUNTS_FILE, "a") as file:
        file.write(f"{account['user_name']},{account['account_title']},{account['account_number']},{account['balance']}\n")

In [137]:
def load_user(account_no_filter):
    if not os.path.exists(ACCOUNTS_FILE):
        print("No user data found.")
        return None

    with open(ACCOUNTS_FILE, "r") as file:
        users = file.readlines()
        for user in users:
            user_name, account_title, account_number, balance = user.strip().split(",")
            if account_no_filter == account_number:
                # print(f"Account Title: {title} (CNIC: {cnic}) loaded with balance ${balance}.")
                return {
                    "user_name": user_name,
                    "account_title": account_title,
                    "account_number": account_number,
                    "balance": float(balance)
                }
    print(f"No record found with Account Number {account_no_filter}.")
    return None

In [138]:
# Deposit function
def deposit(account, amount):
    if amount <= 0:
        print("Error: Deposit amount must be positive.")
        return account
    
    account["balance"] += amount
    record_transaction(account["account_number"], "Deposit", amount, account["balance"])
    update_user_balance(account["account_number"], account["balance"])
    print(f"====== {account['account_title']} deposited ${amount}. New balance: ${account['balance']:.2f}. ======")
    return account

In [139]:
# Record Transaction to file.
# transaction_type: Deposit, Withdrawal
def record_transaction(account_number, transaction_type, amount, balance):
    with open(TRANSACTION_FILE, "a") as file:
        file.write(f"{account_number},{transaction_type},{amount},{balance}\n")

In [140]:
# Update balance in USERS.txt
def update_user_balance(account_no_filter, new_balance):
    if not os.path.exists(USER_FILE):
        print("User list file not found.")
        return

    with open(ACCOUNTS_FILE, "r") as file:
        users = file.readlines()

    updated_users = []
    for user in users:
        user_name, account_title, account_number, balance = user.strip().split(",")
        if account_no_filter == account_number:
            updated_users.append(f"{user_name},{account_title},{account_number},{new_balance:.2f}\n")
        else:
            updated_users.append(user)

    with open(ACCOUNTS_FILE, "w") as file:
        file.writelines(updated_users)

In [141]:
# Withdraw function
def withdraw(account, amount):
    if amount <= 0:
        print("Error: Withdrawal amount must be positive.")
        return account
    
    if amount > account["balance"]:
        print("Error: Insufficient balance.")
        return account
    
    account["balance"] -= amount
    record_transaction(account["account_number"], "Withdrawal", amount, account["balance"])
    update_user_balance(account["account_number"], account["balance"])
    print(f"====== {account['account_title']} withdrew ${amount}. New balance: ${account['balance']:.2f}. ======")
    return account

In [148]:
# Load Statement Function
def load_statement(account):
    if not os.path.exists(TRANSACTION_FILE):
        print("No transactions data found.")
        return None
        
    with open(TRANSACTION_FILE, "r") as file:
        transactions = file.readlines()
        user_transactions = []
        for transaction in transactions:
            account_number, transaction_type, amount, balance = transaction.strip().split(",")
            if account['account_number'] == account_number:
                user_transactions.append({
                    "transaction_type": transaction_type,
                    "amount": float(amount),
                    "balance": float(balance)
                })
        if not user_transactions:
            print(f"No transactions found for account holder {account['account_title']}.")
            return None
        else:
            print(f"\nTransactions for {account['account_title']}:")
            
            for transaction in user_transactions:
                print(f"{transaction['transaction_type']} - Amount: {transaction['amount']} - Balance: {transaction['balance']}")

In [143]:
def load_accounts():
    if not os.path.exists(ACCOUNTS_FILE):
        print("No accounts data found.")
        return None

    with open(ACCOUNTS_FILE, "r") as file:
        users = file.readlines()
        for user in users:
            user_name, account_title, account_number, balance = user.strip().split(",")
            print(f"{user_name} - {account_title} - {account_number} - {balance}")

In [144]:
def load_deposit_transactions():
    if not os.path.exists(TRANSACTION_FILE):
        print("No accounts data found.")
        return None

    with open(TRANSACTION_FILE, "r") as file:
        transactions = file.readlines()
        deposit_transactions = []
        for transaction in transactions:
            account_number, transaction_type, amount, balance = transaction.strip().split(",")
            if transaction_type.lower() == 'deposit':
                deposit_transactions.append({
                        "account_number": account_number,
                        "transaction_type": transaction_type,
                        "amount": float(amount)
                    })
        if not deposit_transactions:
            print(f"No transactions found.")
            return None
        else:
            for transaction in deposit_transactions:
                print(f"{transaction['account_number']} - {transaction['transaction_type']} - Amount: {transaction['amount']}")

In [145]:
def load_withdrawal_transactions():
    if not os.path.exists(TRANSACTION_FILE):
        print("No accounts data found.")
        return None

    with open(TRANSACTION_FILE, "r") as file:
        transactions = file.readlines()
        withdrawal_transactions = []
        for transaction in transactions:
            account_number, transaction_type, amount, balance = transaction.strip().split(",")
            if transaction_type.lower() == 'withdrawal':
                withdrawal_transactions.append({
                        "account_number": account_number,
                        "transaction_type": transaction_type,
                        "amount": float(amount)
                    })
        if not withdrawal_transactions:
            print(f"No transactions found.")
            return None
        else:
            for transaction in withdrawal_transactions:
                print(f"{transaction['account_number']} - {transaction['transaction_type']} - Amount: {transaction['amount']}")

In [149]:
main()


1 > Continue to Login
2 > Exit



Welcome to Banking System. Do you want to Continue? (Y to Continue) (N to Exit) 1
Enter UserName:  usman
Enter Password:  usman



/Dashboard
***** Hello usman - USER *****
1 > Check Balance
2 > Deposit Amount
3 > Withdraw Amount
4 > Statement
5 > Exit



Select what operation to perform:  4


**** /Dashboard/Statement ****


...


Transactions for Muhammad Usman:
Deposit - Amount: 50.0 - Balance: 50.0
Deposit - Amount: 25.0 - Balance: 75.0
Withdrawal - Amount: 10.0 - Balance: 65.0
Deposit - Amount: 20.0 - Balance: 85.0

/Dashboard
***** Hello usman - USER *****
1 > Check Balance
2 > Deposit Amount
3 > Withdraw Amount
4 > Statement
5 > Exit



Select what operation to perform:  5


Exit

1 > Continue to Login
2 > Exit



Welcome to Banking System. Do you want to Continue? (Y to Continue) (N to Exit) 2


Quiting System.
